In [ ]:
!pip install kaggle

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Rescaling
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
!kaggle datasets download -d mohamedmustafa/real-life-violence-situations-dataset


Dataset URL: https://www.kaggle.com/datasets/mohamedmustafa/real-life-violence-situations-dataset
License(s): copyright-authors
100% 3.58G/3.58G [02:41<00:00, 24.6MB/s]
100% 3.58G/3.58G [02:41<00:00, 23.8MB/s]


In [ ]:
!unzip real-life-violence-situations-dataset.zip

Archive:  real-life-violence-situations-dataset.zip
replace Real Life Violence Dataset/NonViolence/NV_1.mp4? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout
import numpy as np
import cv2
import os

# Parameters
target_height = 80
target_width = 80
num_frames = 16
batch_size = 32
epochs = 25

# Function to extract frames from videos
def extract_frames(video_path, target_height, target_width, num_frames):
    print(f"Extracting frames from: {video_path}")
    frames = []
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total frames in video: {total_frames}")
    frame_indices = np.linspace(0, total_frames - 1, num=num_frames, dtype=int)
    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (target_width, target_height))
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
    cap.release()
    print(f"Extracted {len(frames)} frames.")
    return frames

# Function to preprocess videos for a class
def preprocess_videos(data_dir, target_height, target_width, num_frames, label, max_videos=800):
    print(f"Preprocessing videos in directory: {data_dir}")
    X, y = [], []
    count = 0
    for video_name in os.listdir(data_dir):
        count += 1
        if count > max_videos:
            print(f"Reached max_videos limit ({max_videos}). Stopping.")
            break
        video_path = os.path.join(data_dir, video_name)
        print(f"Processing video {count}: {video_name}")
        frames = extract_frames(video_path, target_height, target_width, num_frames)
        if len(frames) == num_frames:
            X.append(frames)
            y.append(label)
        else:
            print(f"Skipping video {video_name} due to insufficient frames.")
    print(f"Processed {count} videos from {data_dir}.")
    return np.array(X), np.array(y)

# Step 1: Preprocess the dataset
data_dir_nonviolence = '/content/Real Life Violence Dataset/NonViolence'
data_dir_violence = '/content/Real Life Violence Dataset/Violence'

# Preprocess NonViolence videos
print("Starting preprocessing of NonViolence videos...")
X_nonviolence, y_nonviolence = preprocess_videos(
    data_dir_nonviolence, target_height, target_width, num_frames, label=0)

# Preprocess Violence videos
print("Starting preprocessing of Violence videos...")
X_violence, y_violence = preprocess_videos(
    data_dir_violence, target_height, target_width, num_frames, label=1)

# Combine features and labels
print("Combining datasets...")
X = np.concatenate((X_nonviolence, X_violence), axis=0)
y = np.concatenate((y_nonviolence, y_violence), axis=0)

# Shuffle the dataset
print("Shuffling the dataset...")
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X = X[indices]
y = y[indices]

# Normalize the data
print("Normalizing the data...")
X = X / 255.0

# Step 2: Define the 3D CNN model
print("Defining the 3D CNN model...")
model = Sequential([
    Conv3D(32, (3, 3, 3), activation='relu', input_shape=(num_frames, target_height, target_width, 3)),
    MaxPooling3D((2, 2, 2)),
    Conv3D(64, (3, 3, 3), activation='relu'),
    MaxPooling3D((2, 2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
print("Compiling the model...")
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Step 3: Train the model
print("Starting training...")
history = model.fit(X, y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

# Step 4: Save the trained model
print("Saving the trained model...")
model.save('/content/violence_detection_model.h5')
print("Model saved successfully.")

# Step 5: Evaluate the model (Optional)
print("Evaluating the model...")
loss, accuracy = model.evaluate(X, y, verbose=1)
print(f"Model Accuracy: {accuracy * 100:.2f}%")


Starting preprocessing of NonViolence videos...
Preprocessing videos in directory: /content/Real Life Violence Dataset/NonViolence
Processing video 1: NV_901.avi
Extracting frames from: /content/Real Life Violence Dataset/NonViolence/NV_901.avi
Total frames in video: 150
Extracted 16 frames.
Processing video 2: NV_421.mp4
Extracting frames from: /content/Real Life Violence Dataset/NonViolence/NV_421.mp4
Total frames in video: 55
Extracted 16 frames.
Processing video 3: NV_249.mp4
Extracting frames from: /content/Real Life Violence Dataset/NonViolence/NV_249.mp4
Total frames in video: 125
Extracted 16 frames.
Processing video 4: NV_973.mp4
Extracting frames from: /content/Real Life Violence Dataset/NonViolence/NV_973.mp4
Total frames in video: 145
Extracted 16 frames.
Processing video 5: NV_156.mp4
Extracting frames from: /content/Real Life Violence Dataset/NonViolence/NV_156.mp4
Total frames in video: 125
Extracted 16 frames.
Processing video 6: NV_337.mp4
Extracting frames from: /cont

KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!rsync -avzh  '/content/violence_detection_model.h5'  '/content/drive/MyDrive/Colab Notebooks'

sending incremental file list
violence_detection_model.h5

sent 53.20M bytes  received 35 bytes  35.47M bytes/sec
total size is 64.44M  speedup is 1.21



```
`**#For test the code**`
```[link text](https://)



In [ ]:
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('/content/violence_detection_model.h5')
print("Model loaded successfully.")


Model loaded successfully.


In [ ]:
def preprocess_input_video(video_path):
    frames = extract_frames(video_path, target_height=80, target_width=80, num_frames=16)
    if len(frames) == 16:  # Ensure we have the required number of frames
        frames = np.array(frames) / 255.0  # Normalize
        return np.expand_dims(frames, axis=0)  # Add batch dimension
    else:
        print("Error: Video does not have enough frames.")
        return None


In [ ]:
# Path to your input video
input_video_path = '/content/real life violence situations/Real Life Violence Dataset/Violence/V_1.mp4'

# Preprocess the video
input_data = preprocess_input_video(input_video_path)

if input_data is not None:
    # Make predictions
    prediction = model.predict(input_data)
    print(f"Prediction: {prediction[0][0]:.4f}")

    # Interpret the result
    if prediction[0][0] > 0.5:
        print("The video is classified as: Violence")
    else:
        print("The video is classified as: Non-Violence")


Extracting frames from: /content/real life violence situations/Real Life Violence Dataset/Violence/V_1.mp4
Total frames in video: 103
Extracted 16 frames.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 781ms/step
Prediction: 0.9999
The video is classified as: Violence


In [ ]:
import os

# Directory containing videos for batch processing
video_dir = '/content/real life violence situations/Real Life Violence Dataset/Violence'

def batch_process_videos(video_dir, model):
    print(f"Processing all videos in directory: {video_dir}")
    results = {}
    for video_file in os.listdir(video_dir):
        if video_file.endswith(('.mp4', '.avi', '.mkv')):  # Ensure it's a video file
            video_path = os.path.join(video_dir, video_file)
            print(f"Processing video: {video_file}")
            input_data = preprocess_input_video(video_path)
            if input_data is not None:
                # Make predictions
                prediction = model.predict(input_data)
                label = 'Violence' if prediction[0][0] > 0.5 else 'Non-Violence'
                score = prediction[0][0]
                results[video_file] = {'label': label, 'score': float(score)}
                print(f"{video_file}: {label} (Score: {score:.4f})")
            else:
                print(f"Skipping {video_file}: Not enough frames.")
    return results

# Call batch processing
batch_results = batch_process_videos(video_dir, model)

# Print final results
print("\nBatch Processing Results:")
for video, result in batch_results.items():
    print(f"{video}: {result['label']} (Score: {result['score']:.4f})")


Streaming output truncated to the last 5000 lines.
Extracted 16 frames.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
V_726.mp4: Violence (Score: 0.9998)
Processing video: V_485.mp4
Extracting frames from: /content/real life violence situations/Real Life Violence Dataset/Violence/V_485.mp4
Total frames in video: 153
Extracted 16 frames.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
V_485.mp4: Violence (Score: 0.9998)
Processing video: V_725.mp4
Extracting frames from: /content/real life violence situations/Real Life Violence Dataset/Violence/V_725.mp4
Total frames in video: 115
Extracted 16 frames.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
V_725.mp4: Violence (Score: 1.0000)
Processing video: V_12.mp4
Extracting frames from: /content/real life violence situations/Real Life Violence Dataset/Violence/V_12.mp4
Total frames in video: 138
Extracted 16 frames.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
V_12.mp4: Violence (Score: 1.0000)
Processing video: V_833.mp4
Extracting frames from: /content/real life violence 

In [ ]:
!pip install Flask
!pip install pyngrok

Bulk data in case of folder

**web interface using flask**

In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

# Load the trained model
from tensorflow.keras.models import load_model
model = load_model('/content/violence_detection_model.h5')

@app.route('/predict', methods=['POST'])
def predict():
    video_path = request.json.get('video_path')
    if not video_path:
        return jsonify({'error': 'No video path provided'}), 400

    # Preprocess the video
    input_data = preprocess_input_video(video_path)
    if input_data is None:
        return jsonify({'error': 'Video processing failed. Ensure the video has sufficient frames.'}), 400

    # Make predictions
    prediction = model.predict(input_data)
    label = 'Violence' if prediction[0][0] > 0.5 else 'Non-Violence'
    score = float(prediction[0][0])

    return jsonify({'label': label, 'score': score})

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
